In [37]:
import string
import sys
import os
import pandas as pd
import numpy as np
import re
from tqdm import *
tqdm.pandas()

In [38]:
Names_new= pd.read_csv("/Users/kamakshibansal/Moorfields/Distinct_name_new_count.csv")
Names=pd.DataFrame()
Names["Forename"]=Names_new["Forename"]
Names=Names.dropna(axis=0)
wordFrequency_name = dict.fromkeys(Names["Forename"].str.lower().unique().tolist(),1 )

In [39]:
Names

,Forename
0,adlane
1,admira
2,adotey
3,adrews
4,adzewnko
...,...
184529,david
184530,lane
184531,essex
184532,john


In [40]:
def clean_sentences(sentence):
    result1= re.sub(r"""["?,>}{)"\/(.\\:$!]|'(?!(?<! ')[ts])|'s""", " ", sentence.lower())
    result1= re.sub(' +', ' ', result1)
    result1= re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '',result1)
    result2= remove_hospitalname(result1)
    return result2

def clean_mr(query):
    mr_list = re.compile(r'(^ms|^ms.|^miss|^mr|^mr.|^mrs.|^miss.|^mrs|^dr.|^dr|^dear.|^dob|^number|\s*ms|\s*ms.|\s*miss|\s*mr|\s*mr.|\s*mrs.|\s*miss.|\s*mrs|\s*dr.|\s*dr|\s*dob|\s*number)\s+([^\s]+)\s*').findall(query)
    mr_list = sorted(mr_list, key = lambda k : len(k[1]), reverse=True)
    for mr_word_tuple in mr_list:
        word_comb = ' '.join(list(mr_word_tuple))
        first_word = mr_word_tuple[0]
        query = query.replace(word_comb, first_word+' ')
        
    query=re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '',query)
    return query

def clean_mr_and_remove_stopwords(query, merge_dict):
    result2 = clean_mr(query).split()
    resultwords  = [word for word in result2 if merge_dict[word]==0]
    final1= ' '.join(resultwords) 
    final2= remove_match(final1)
    final= remove_cc(final2)
#     print (final)
    return final

def remove_match(qu):
    qu= re.sub('re.*b0','XXXXXXXX', qu, flags= re.DOTALL)
    return qu

def remove_hospitalname(abc):
    result= re.sub(r"\w+(?=\s+hospital)", "", abc)
    return result

def remove_cc(qu):
    result= re.sub('original recipient.*par','XXXXXXXX', qu, flags= re.DOTALL)
    return result

In [21]:
# for i in range(1999, 2020):
#     print(i)

In [41]:
# number_of_files = 2

for i in range(2010, 2011):
    print ("Working on year {} file".format(i))
    Letters = pd.read_csv("/Users/kamakshibansal/Moorfields/Letters/Cleaned_Letters/cleaned{}.csv".format(i))
    Letters=Letters.dropna()


    b=Letters["LetterBody"].map(clean_sentences).str.lower()
    b=b.values.tolist()
    k=' '.join(b)
    z=k.split()

    z1=list(dict.fromkeys(z))
    wordFrequency = dict.fromkeys(z1,0 )
    merge_dict= { **wordFrequency,**wordFrequency_name}

    %timeit -n 1 -r 1 Letters['clean_sentence'] = Letters.LetterBody.progress_map(clean_sentences)
    %timeit -n 1 -r 1 Letters['final_sentence'] = Letters.clean_sentence.progress_map(lambda c: clean_mr_and_remove_stopwords(c, merge_dict))

    Letters= Letters.drop(["PatKey","clean_sentence", "SeqId","LetterBodyX"], axis=1)

    Letters.to_csv("/Users/kamakshibansal/Moorfields/Letters/Anonymised/Letters_new_{}.csv".format(i))
    


Working on year 2010 file












  0%|          | 0/250374 [00:00<?, ?it/s]









  0%|          | 219/250374 [00:00<01:55, 2174.69it/s]









  0%|          | 454/250374 [00:00<01:52, 2221.56it/s]









  0%|          | 737/250374 [00:00<01:45, 2373.67it/s]









  0%|          | 984/250374 [00:00<01:43, 2401.29it/s]









  1%|          | 1275/250374 [00:00<01:38, 2533.64it/s]









  1%|          | 1555/250374 [00:00<01:35, 2606.44it/s]









  1%|          | 1827/250374 [00:00<01:34, 2639.46it/s]









  1%|          | 2118/250374 [00:00<01:31, 2714.85it/s]









  1%|          | 2405/250374 [00:00<01:29, 2758.31it/s]









  1%|          | 2688/250374 [00:01<01:29, 2777.23it/s]









  1%|          | 2981/250374 [00:01<01:27, 2819.65it/s]









  1%|▏         | 3276/250374 [00:01<01:26, 2856.42it/s]









  1%|▏         | 3560/250374 [00:01<01:26, 2837.46it/s]









  2%|▏         | 3848/250374 [00:01<01:26, 2849.88it/s]









  2%|▏         | 4134/250374 [00

 10%|▉         | 24722/250374 [00:08<01:16, 2964.47it/s]









 10%|▉         | 25022/250374 [00:08<01:15, 2972.82it/s]









 10%|█         | 25328/250374 [00:08<01:15, 2996.94it/s]









 10%|█         | 25637/250374 [00:08<01:14, 3022.14it/s]









 10%|█         | 25940/250374 [00:09<01:14, 3018.49it/s]









 10%|█         | 26261/250374 [00:09<01:12, 3072.94it/s]









 11%|█         | 26586/250374 [00:09<01:11, 3123.95it/s]









 11%|█         | 26899/250374 [00:09<01:15, 2973.81it/s]









 11%|█         | 27217/250374 [00:09<01:13, 3032.01it/s]









 11%|█         | 27522/250374 [00:09<01:13, 3035.20it/s]









 11%|█         | 27842/250374 [00:09<01:12, 3079.88it/s]









 11%|█▏        | 28172/250374 [00:09<01:10, 3141.73it/s]









 11%|█▏        | 28488/250374 [00:09<01:10, 3134.00it/s]









 12%|█▏        | 28810/250374 [00:09<01:10, 3158.54it/s]









 12%|█▏        | 29127/250374 [00:10<01:10, 3133.20it/s]









 12%|█▏   

 21%|██        | 51499/250374 [00:17<01:06, 3008.59it/s]









 21%|██        | 51807/250374 [00:17<01:05, 3029.45it/s]









 21%|██        | 52116/250374 [00:17<01:05, 3046.30it/s]









 21%|██        | 52421/250374 [00:17<01:05, 3032.35it/s]









 21%|██        | 52742/250374 [00:17<01:04, 3081.48it/s]









 21%|██        | 53051/250374 [00:17<01:05, 2994.86it/s]









 21%|██▏       | 53365/250374 [00:17<01:04, 3034.48it/s]









 21%|██▏       | 53684/250374 [00:18<01:03, 3078.69it/s]









 22%|██▏       | 54002/250374 [00:18<01:03, 3106.92it/s]









 22%|██▏       | 54314/250374 [00:18<01:03, 3086.35it/s]









 22%|██▏       | 54635/250374 [00:18<01:02, 3122.14it/s]









 22%|██▏       | 54948/250374 [00:18<01:03, 3070.06it/s]









 22%|██▏       | 55256/250374 [00:18<01:07, 2872.56it/s]









 22%|██▏       | 55549/250374 [00:18<01:07, 2888.38it/s]









 22%|██▏       | 55856/250374 [00:18<01:06, 2940.25it/s]









 22%|██▏  

 30%|███       | 75891/250374 [00:26<01:07, 2598.56it/s]









 30%|███       | 76178/250374 [00:26<01:05, 2674.17it/s]









 31%|███       | 76447/250374 [00:26<01:05, 2672.40it/s]









 31%|███       | 76716/250374 [00:26<01:04, 2677.16it/s]









 31%|███       | 76995/250374 [00:26<01:04, 2707.46it/s]









 31%|███       | 77267/250374 [00:26<01:04, 2688.22it/s]









 31%|███       | 77541/250374 [00:26<01:03, 2701.43it/s]









 31%|███       | 77812/250374 [00:26<01:05, 2637.93it/s]









 31%|███       | 78077/250374 [00:26<01:06, 2576.46it/s]









 31%|███▏      | 78342/250374 [00:27<01:06, 2597.91it/s]









 31%|███▏      | 78603/250374 [00:27<01:06, 2600.38it/s]









 32%|███▏      | 78870/250374 [00:27<01:05, 2620.49it/s]









 32%|███▏      | 79133/250374 [00:27<01:07, 2544.64it/s]









 32%|███▏      | 79389/250374 [00:27<01:08, 2509.87it/s]









 32%|███▏      | 79641/250374 [00:27<01:08, 2474.85it/s]









 32%|███▏ 

...

 81%|████████  | 202920/250374 [02:10<00:31, 1514.06it/s]









 81%|████████  | 203077/250374 [02:10<00:30, 1529.84it/s]









 81%|████████  | 203242/250374 [02:10<00:30, 1563.14it/s]









 81%|████████  | 203401/250374 [02:10<00:29, 1571.02it/s]









 81%|████████▏ | 203565/250374 [02:10<00:29, 1589.74it/s]









 81%|████████▏ | 203725/250374 [02:10<00:29, 1583.29it/s]









 81%|████████▏ | 203884/250374 [02:10<00:30, 1517.31it/s]









 81%|████████▏ | 204037/250374 [02:11<00:31, 1477.25it/s]









 82%|████████▏ | 204186/250374 [02:11<00:31, 1453.50it/s]









 82%|████████▏ | 204344/250374 [02:11<00:30, 1489.21it/s]









 82%|████████▏ | 204509/250374 [02:11<00:29, 1532.96it/s]









 82%|████████▏ | 204664/250374 [02:11<00:29, 1533.52it/s]









 82%|████████▏ | 204819/250374 [02:11<00:29, 1537.74it/s]









 82%|████████▏ | 204981/250374 [02:11<00:29, 1559.79it/s]









 82%|████████▏ | 205139/250374 [02:11<00:28, 1564.18it/s]






 86%|████████▋ | 216368/250374 [02:18<00:22, 1489.26it/s]









 86%|████████▋ | 216518/250374 [02:19<00:23, 1437.69it/s]









 87%|████████▋ | 216668/250374 [02:19<00:23, 1454.61it/s]









 87%|████████▋ | 216814/250374 [02:19<00:23, 1433.20it/s]









 87%|████████▋ | 216976/250374 [02:19<00:22, 1482.20it/s]









 87%|████████▋ | 217125/250374 [02:19<00:22, 1473.47it/s]









 87%|████████▋ | 217284/250374 [02:19<00:21, 1505.64it/s]









 87%|████████▋ | 217445/250374 [02:19<00:21, 1535.26it/s]









 87%|████████▋ | 217600/250374 [02:19<00:21, 1525.00it/s]









 87%|████████▋ | 217758/250374 [02:19<00:21, 1539.15it/s]









 87%|████████▋ | 217913/250374 [02:20<00:23, 1381.73it/s]









 87%|████████▋ | 218063/250374 [02:20<00:22, 1413.96it/s]









 87%|████████▋ | 218212/250374 [02:20<00:22, 1435.41it/s]









 87%|████████▋ | 218380/250374 [02:20<00:21, 1499.97it/s]









 87%|████████▋ | 218537/250374 [02:20<00:20, 1520.07it/s]






 92%|█████████▏| 229931/250374 [02:27<00:13, 1551.29it/s]









 92%|█████████▏| 230087/250374 [02:27<00:13, 1497.31it/s]









 92%|█████████▏| 230249/250374 [02:27<00:13, 1530.63it/s]









 92%|█████████▏| 230403/250374 [02:27<00:13, 1489.02it/s]









 92%|█████████▏| 230555/250374 [02:28<00:13, 1497.17it/s]









 92%|█████████▏| 230706/250374 [02:28<00:13, 1476.73it/s]









 92%|█████████▏| 230859/250374 [02:28<00:13, 1490.47it/s]









 92%|█████████▏| 231015/250374 [02:28<00:12, 1509.12it/s]









 92%|█████████▏| 231167/250374 [02:28<00:12, 1510.08it/s]









 92%|█████████▏| 231342/250374 [02:28<00:12, 1573.16it/s]









 92%|█████████▏| 231511/250374 [02:28<00:11, 1604.98it/s]









 93%|█████████▎| 231673/250374 [02:28<00:11, 1593.36it/s]









 93%|█████████▎| 231833/250374 [02:28<00:11, 1569.02it/s]









 93%|█████████▎| 231999/250374 [02:29<00:11, 1592.99it/s]









 93%|█████████▎| 232164/250374 [02:29<00:11, 1608.06it/s]






 97%|█████████▋| 242633/250374 [02:36<00:05, 1386.32it/s]









 97%|█████████▋| 242778/250374 [02:36<00:05, 1385.54it/s]









 97%|█████████▋| 242920/250374 [02:36<00:05, 1394.32it/s]









 97%|█████████▋| 243067/250374 [02:36<00:05, 1415.17it/s]









 97%|█████████▋| 243217/250374 [02:36<00:04, 1438.08it/s]









 97%|█████████▋| 243362/250374 [02:36<00:04, 1423.74it/s]









 97%|█████████▋| 243505/250374 [02:37<00:04, 1409.64it/s]









 97%|█████████▋| 243647/250374 [02:37<00:04, 1393.73it/s]









 97%|█████████▋| 243787/250374 [02:37<00:04, 1390.96it/s]









 97%|█████████▋| 243927/250374 [02:37<00:04, 1363.31it/s]









 97%|█████████▋| 244064/250374 [02:37<00:04, 1277.63it/s]









 98%|█████████▊| 244202/250374 [02:37<00:04, 1305.45it/s]









 98%|█████████▊| 244334/250374 [02:37<00:04, 1309.39it/s]









 98%|█████████▊| 244478/250374 [02:37<00:04, 1345.58it/s]









 98%|█████████▊| 244623/250374 [02:37<00:04, 1372.98it/s]






2min 42s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
